In [ ]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [ ]:
%%capture
import sys
if 'google.colab' in sys.modules:
    !pip install -Uqq transformers[onnx]

In [ ]:
from huggingface_hub import notebook_login, hf_hub_url

# notebook_login()

In [ ]:
import pandas as pd
import torch
from transformers import AutoTokenizer
from onnxruntime import InferenceSession

In [ ]:
model_id = "arampacha/gpt-neo-therapist-small"
onnx_model_url = hf_hub_url(model_id, filename="model.onnx")

!wget $onnx_model_url

--2022-03-31 19:39:36--  https://huggingface.co/arampacha/gpt-neo-therapist-small/resolve/main/model.onnx
Resolving huggingface.co (huggingface.co)... 34.198.1.82, 34.224.55.150, 34.197.58.156, ...
Connecting to huggingface.co (huggingface.co)|34.198.1.82|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/2b/93/2b9302df8c51888d53898d9367b58ba06e7a96824ace5285f8466212c73342ed/0a67ef944e3d977e6bc86c18633d95904d753f6ad93caa6cffce1b4c7118d87c [following]
--2022-03-31 19:39:36--  https://cdn-lfs.huggingface.co/repos/2b/93/2b9302df8c51888d53898d9367b58ba06e7a96824ace5285f8466212c73342ed/0a67ef944e3d977e6bc86c18633d95904d753f6ad93caa6cffce1b4c7118d87c
Resolving cdn-lfs.huggingface.co (cdn-lfs.huggingface.co)... 13.225.142.70, 13.225.142.103, 13.225.142.84, ...
Connecting to cdn-lfs.huggingface.co (cdn-lfs.huggingface.co)|13.225.142.70|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 705611856 (673M) [app

In [ ]:
# !wget https://raw.githubusercontent.com/nbertagnolli/counsel-chat/master/data/20200325_counsel_chat.

df = pd.read_csv("20200325_counsel_chat.csv")
df = df[df.split=="test"]

In [ ]:
import random

i = random.randint(0, len(df)-1)
sample = df.iloc[i, :]

prompt_text = f"Answer like a therapist:\n {sample.questionTitle} {sample.questionText}\nAnswer: "
ref_answer = sample.answerText

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("arampacha/gpt-neo-therapist-small")#, use_auth_token=True)
# session = InferenceSession("model-quantized.onnx")

Downloading:   0%|          | 0.00/615 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/779k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.01M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/387 [00:00<?, ?B/s]

In [ ]:
import torch
from transformers.modeling_outputs import CausalLMOutputWithPast
from transformers.generation_utils import GenerationMixin
import numpy as np

from typing import Dict


class DummySession:
    def __init__(self) -> None:
        pass
        
    def run(self, output_names, input_feed):
        input_shape = input_feed["input_ids"].shape
        bs, nh, sl, d = input_feed["past_key_values.0.key"].shape 
        for v in input_feed.values():
            assert isinstance(v, np.ndarray)
        logit_shape = input_shape+(10,)
        fake_logits = np.random.randn(*logit_shape)
        fake_past = [np.random.randn(bs, nh, sl+1, d) for i in range(len(input_feed)-2)]
        return [fake_logits] + fake_past

def to_numpy(x):
    if isinstance(x, np.ndarray):
        return x
    if isinstance(x, torch.Tensor):
        return x.cpu().numpy()
    if isinstance(x, list):
        return [to_numpy(e) for e in x]
    if isinstance(x, dict):
        return {k:to_numpy(v) for k, v in x.items()}

def to_pt(x):
    if isinstance(x, torch.Tensor):
        return x
    if isinstance(x, np.ndarray):
        return torch.tensor(x)
    if isinstance(x, list):
        return [to_pt(e) for e in x]
    if isinstance(x, dict):
        return {k:to_pt(v) for k, v in x.items()}


class ONNXWrapper(GenerationMixin):

    def __init__(self, onnx_model, config, device="cpu") -> None:
        self.session = InferenceSession(onnx_model)
        self.config = config
        self.main_input_name = "input_ids"
        self.device = torch.device(device)
        #TODO: check config
        self.output_names = ["logits"] + [f"present.{i//2}.{'key' if i%2==0 else 'value'}" for i in range(self.config.num_layers*2)]
        self.past_keys = [f"past_key_values.{i//2}.{'key' if i%2==0 else 'value'}" for i in range(config.num_layers*2)]

    def forward(input_ids, attention_mask, past_key_values):
        # dummy method for compatibility
        pass

    def get_empty_past(self, input_ids, config):
        batch_size = input_ids.shape[0]
        past_shape = [batch_size, config.num_heads, 0, config.hidden_size // config.num_heads]
        empty_past = {k: np.empty(past_shape, np.float32) for k in self.past_keys}
        return empty_past

    def prepare_inputs_for_generation(self, input_ids, past=None, **kwargs):
        if past is not None:
            input_ids = input_ids[:, -1:]
        else:
            past = self.get_empty_past(input_ids, self.config)

        attention_mask = kwargs.get("attention_mask", None)
        position_ids = kwargs.get("position_ids", None)
        if attention_mask is not None and position_ids is None:
            # create position_ids on the fly for batch generation
            position_ids = attention_mask.long().cumsum(-1) - 1
            position_ids.masked_fill_(attention_mask == 0, 1)
            if past:
                position_ids = position_ids[:, -1:]
        else:
            position_ids = None
        # position_ids not used by ONNX model
        return {
            "inputs":{
                "input_ids": input_ids,
                "attention_mask": attention_mask
            },
            "past":past
        }
    
    @staticmethod
    def _update_model_kwargs_for_generation(outputs, model_kwargs, is_encoder_decoder):
        # update past
        if "past_key_values" in outputs:
            model_kwargs["past"] = outputs.past_key_values
        else:
            model_kwargs["past"] = None

        # update token_type_ids with last value
        if "token_type_ids" in model_kwargs:
            token_type_ids = model_kwargs["token_type_ids"]
            model_kwargs["token_type_ids"] = torch.cat([token_type_ids, token_type_ids[:, -1].unsqueeze(-1)], dim=-1)

        # update attention mask
        if "attention_mask" in model_kwargs:
            attention_mask = model_kwargs["attention_mask"]
            model_kwargs["attention_mask"] = torch.cat(
                [attention_mask, attention_mask.new_ones((attention_mask.shape[0], 1))], dim=-1
            )
        return model_kwargs

    def __call__(self, inputs, past=None, **kwargs) -> CausalLMOutputWithPast:
        
        inputs["attention_mask"] = inputs["attention_mask"].float() #.astype(np.float32)
        # print(inputs)
        # print(past)
        outputs = to_pt(self.session.run(output_names=self.output_names, input_feed={**to_numpy(inputs), **to_numpy(past)}))
        
        return CausalLMOutputWithPast(logits=outputs[0], past_key_values={k:v for k,v in zip(self.past_keys, outputs[1:])})


In [ ]:
from transformers import AutoConfig

config = AutoConfig.from_pretrained(model_id)
model = ONNXWrapper("model.onnx", config)

In [ ]:
inputs = tokenizer(prompt_text, return_tensors="pt")
prompt_length = inputs["input_ids"].size(1)
# outputs = session.run(output_names=["logits", "past_key_values"], input_feed=dict(inputs))

output_ids = model.generate(inputs["input_ids"], do_sample=True, top_p=0.9, max_length=prompt_length+200)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [ ]:
tokenizer.decode(output_ids.squeeze()[prompt_length:], skip_specail_tokens=True).strip()

"My boyfriend doesn't know why he thinks he's doing what he is doing. In most ways he's not even talking about anything negative. You don't get to see him for what he is, except for his words. You are talking in the beginning, but we aren't sure what to do with him. He would have to find out, then you'll learn it would be better for him to say a few things to his wife. Maybe I'm not supposed to know what he is going"

In [ ]:
model_q = ONNXWrapper("model-quantized.onnx", config)

In [ ]:
inputs = tokenizer(prompt_text, return_tensors="pt")
prompt_length = inputs["input_ids"].size(1)
# outputs = session.run(output_names=["logits", "past_key_values"], input_feed=dict(inputs))

output_ids = model_q.generate(inputs["input_ids"], do_sample=True, top_p=0.9, max_length=prompt_length+200)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [ ]:
tokenizer.decode(output_ids.squeeze()[prompt_length:], skip_specail_tokens=True).strip()

"That it is odd how many of these kids are actually trying to get a good answer out of them. So they want to hear and be told:\nDoes it have something else going on? Because if it is, it is not good. If we are telling you that's fine. But if we are not telling you of any other things that are important, of all times, how would you know what that answer sounds good? What we are saying is, you can't even imagine what that is. That's wrong. It is not important. We are saying this is a very important topic, that is going to be very important, and we are going to start off with this. What are we saying? Because if it is not important, we are not going to tell people that they are not going to hear. You see these problems in your life and I am not saying that we are not going to think about that. People are not going to talk about these issues. When we"